In [1]:
import pandas as pd 
import matplotlib.pyplot as plt  

In [2]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split

In [4]:
df=pd.read_csv("credit.csv")
df

,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_duration,percent_of_income,years_at_residence,age,other_credit,housing,existing_loans_count,job,dependents,phone,default
0,< 0 DM,6,critical,furniture/appliances,1169,unknown,> 7 years,4,4,67,none,own,2,skilled,1,yes,no
1,1 - 200 DM,48,good,furniture/appliances,5951,< 100 DM,1 - 4 years,2,2,22,none,own,1,skilled,1,no,yes
2,unknown,12,critical,education,2096,< 100 DM,4 - 7 years,2,3,49,none,own,1,unskilled,2,no,no
3,< 0 DM,42,good,furniture/appliances,7882,< 100 DM,4 - 7 years,2,4,45,none,other,1,skilled,2,no,no
4,< 0 DM,24,poor,car,4870,< 100 DM,1 - 4 years,3,4,53,none,other,2,skilled,2,no,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,unknown,12,good,furniture/appliances,1736,< 100 DM,4 - 7 years,3,4,31,none,own,1,unskilled,1,no,no
996,< 0 DM,30,good,car,3857,< 100 DM,1 - 4 years,4,4,40,none,own,1,management,1,yes,no
997,unknown,12,good,furniture/appliances,804,< 100 DM,> 7 years,4,4,38,none,own,1,skilled,1,no,no
998,< 0 DM,45,good,furniture/appliances,1845,< 100 DM,1 - 4 years,4,4,23,none,other,1,skilled,1,yes,yes


# Data cleaning and preprocesing 

In [5]:
lb = LabelEncoder() 

In [6]:
df['checking_balance']=lb.fit_transform(df['checking_balance'])
df['credit_history']=lb.fit_transform(df['credit_history'])
df['purpose']=lb.fit_transform(df['purpose'])
df['savings_balance']=lb.fit_transform(df['savings_balance'])
df['employment_duration']=lb.fit_transform(df['employment_duration'])
df['other_credit']=lb.fit_transform(df['other_credit'])
df['housing']=lb.fit_transform(df['housing'])

In [7]:
df['job']=lb.fit_transform(df['job'])

In [8]:
df.sample(5)

,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_duration,percent_of_income,years_at_residence,age,other_credit,housing,existing_loans_count,job,dependents,phone,default
955,1,24,0,4,1231,3,3,4,4,57,1,2,2,0,1,yes,no
97,0,18,1,0,2622,0,0,4,4,34,1,1,1,1,1,no,no
640,1,18,1,3,750,2,4,4,1,27,1,1,1,2,1,no,yes
44,1,48,0,1,6143,2,3,4,4,58,2,0,2,3,1,no,yes
494,1,12,0,1,2122,2,0,3,2,39,1,2,2,3,2,no,no


In [9]:
# checking the non-numercal colums 
non_numerical_cols = df.select_dtypes(include=['object']).columns 
print(non_numerical_cols)

Index(['phone', 'default'], dtype='object')


In [10]:
df['checking_balance'] = lb.fit_transform(df["checking_balance"])
df['default'] = lb.fit_transform(df['default'])

In [11]:
df=df.drop(columns=['phone'])

# Model Bulinding

In [12]:
X=df.iloc[:300,:4]

In [13]:
X.shape

(300, 4)

In [19]:
y=df.iloc[:300,-1]

In [20]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [21]:
# Create a baseline SVM classifier
baseline_svm = SVC()
baseline_svm.fit(X_train, y_train)
y_pred = baseline_svm.predict(X_test)

In [22]:
# Evaluate the baseline model
accuracy = accuracy_score(y_test, y_pred)
print(f"Baseline SVM Accuracy: {accuracy:.2f}")

Baseline SVM Accuracy: 0.72


In [23]:
from sklearn.model_selection import GridSearchCV

In [24]:
# Define the hyperparameter grid
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': [0.1, 1, 'scale', 'auto']
}

In [ ]:
# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=baseline_svm, param_grid=param_grid, cv=5)


# Fit the model with the grid of hyperparameters
grid_search.fit(X_train, y_train)

In [ ]:
# Get the best hyperparameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_


y_pred_best = best_model.predict(X_test)
accuracy_best = accuracy_score(y_test, y_pred_best)
print(f"Best SVM Accuracy: {accuracy_best:.2f}")
print(f"Best Hyperparameters: {best_params}")

# Randomized Search

In [15]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
import numpy as np
param_dist = {
    'C': uniform(0.1, 10),  
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto'] + list(np.logspace(-3, 3, 50))
}

In [16]:

randomized_search = RandomizedSearchCV(estimator=baseline_svm, param_distributions=param_dist, n_iter=20, cv=5)

randomized_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=SVC(), n_iter=20,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000002889D5DDFD0>,
                                        'gamma': ['scale', 'auto', 0.001,
                                                  0.0013257113655901094,
                                                  0.0017575106248547913,
                                                  0.002329951810515372,
                                                  0.0030888435964774815,
                                                  0.004094915062380427,
                                                  0.005428675439323859,
                                                  0.0071968567300115215,
                                                  0.009540954...
                                                  0.022229964825261943,
                                                  0.029470517025518096,
                                                  0.03906939937054617,
                                                  0.0517947467923121,
                                                  0.06866488450043001,
                                                  0.09102981779915217,
                                                  0.1206792640639329,
                                                  0.15998587196060574,
                                                  0.21209508879201905,
                                                  0.28117686979742307,
                                                  0.3727593720314938,
                                                  0.49417133613238334,
                                                  0.655128556859551,
                                                  0.868511373751352,
                                                  1.151395399326447,
                                                  1.5264179671752334,
                                                  2.023589647725158, ...],
                                        'kernel': ['linear', 'rbf', 'poly']})

In [17]:
# Get the best hyperparameters and model
best_params_rand = randomized_search.best_params_
best_model_rand = randomized_search.best_estimator_


In [18]:
y_pred_best_rand = best_model_rand.predict(X_test)
accuracy_best_rand = accuracy_score(y_test, y_pred_best_rand)
print(f"Best SVM Accuracy: {accuracy_best_rand:.2f}")
print(f"Best Hyperparameters: {best_params_rand}")

Best SVM Accuracy: 0.94
Best Hyperparameters: {'C': 0.3818470225057634, 'gamma': 0.0013257113655901094, 'kernel': 'poly'}


# Conclusion